In [4]:
%pip install -q gliner glirel spacy loguru huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip show spacy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: spacy
Version: 3.7.5
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /home/donbr/kgmgmt/kg_rememberall/.venv/lib/python3.11/site-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, preshed, pydantic, requests, setuptools, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi, weasel
Required-by: en-core-web-md, gliner-spacy
Note: you may need to restart the kernel to use updated packages.


In [7]:
!python -m spacy download en_core_web_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## initial validation

- appears to be working much better after switch to lower case entity labels and using proper english terms rather than standard NER upper case terms.

- next steps
 - replicate some of the tests referenced in the GLiREL paper
 - co-reference resolution

In [8]:
import os

HF_LOGIN = os.environ.get("HF_LOGIN")

In [9]:
# Import necessary libraries
from gliner import GLiNER  # For Named Entity Recognition
from glirel import GLiREL  # For Relation Extraction
import spacy              # For text processing and tokenization

# Load the full text of Winston Churchill's speech
speech_text = """
From the moment that the French defenses at Sedan and on the Meuse were broken at the end of the second week of May, only a rapid retreat to Amiens and the south could have saved the British and French Armies who had entered Belgium at the appeal of the Belgian King; but this strategic fact was not immediately realized. The French High Command hoped they would be able to close the gap, and the Armies of the north were under their orders. Moreover, a retirement of this kind would have involved almost certainly the destruction of the fine Belgian Army of over 20 divisions and the abandonment of the whole of Belgium. Therefore, when the force and scope of the German penetration were realized and when a new French Generalissimo, General Weygand, assumed command in place of General Gamelin, an effort was made by the French and British Armies in Belgium to keep on holding the right hand of the Belgians and to give their own right hand to a newly created French Army which was to have advanced across the Somme in great strength to grasp it.
"""  # Replace with the complete speech text

# Initialize the GLiNER model for NER
gliner_model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

# Define the entity labels relevant to the speech
entity_labels = [
    "person",         # Winston Churchill, General Weygand
    "military",       # Armies, divisions, forces
    "nation",         # British, French, Belgian
    "location",       # Sedan, Meuse, Amiens, Belgium
    "battle",         # specific military engagements
    "date",          # temporal references
    "military unit",  # divisions, armies
    "rank",          # General, Commander
    "title"          # other formal titles/positions
]

# Perform Named Entity Recognition on the speech text
entities = gliner_model.predict_entities(speech_text, entity_labels, threshold=0.5)

# Display the recognized entities
print("Recognized Entities:")
for entity in entities:
    print(f"Entity: {entity['text']} | Type: {entity['label']} | Span: ({entity['start']}, {entity['end']})")

# Initialize spaCy for tokenization
nlp = spacy.load("en_core_web_sm")
doc = nlp(speech_text)
tokens = [token.text for token in doc]

# Initialize the GLiREL model for Relation Extraction
glirel_model = GLiREL.from_pretrained("jackboyla/glirel-large-v0")

# Define the relation labels to identify
relation_labels = [
    "leads",
    "participates in",
    "located in",
    "defends",
    "opposes",
    "commands",
    "part of",
    "attacks",
    "retreats from",
    "allies with"
]

# Prepare the NER data in the format required by GLiREL
ner_data = []
for entity in entities:
    start_token = len(nlp(speech_text[:entity['start']]))
    end_token = len(nlp(speech_text[:entity['end']]))
    ner_data.append([start_token, end_token, entity['label'], entity['text']])

# Perform Relation Extraction using GLiREL
relations = glirel_model.predict_relations(tokens, relation_labels, threshold=0.5, ner=ner_data, top_k=5)

# Display the extracted relations
print("\nExtracted Relations:")
for relation in relations:
    print(f"Relation: {relation['head_text']} -[{relation['label']}]-> {relation['tail_text']} | Score: {relation['score']}")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/home/donbr/kgmgmt/kg_rememberall/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Recognized Entities:
Entity: Sedan | Type: location | Span: (45, 50)
Entity: Meuse | Type: location | Span: (62, 67)
Entity: second week of May | Type: date | Span: (98, 116)
Entity: Amiens | Type: location | Span: (142, 148)
Entity: British and French Armies | Type: military | Span: (184, 209)
Entity: Belgium | Type: nation | Span: (226, 233)
Entity: Belgian King | Type: person | Span: (255, 267)
Entity: French High Command | Type: military | Span: (327, 346)
Entity: Armies of the north | Type: military unit | Span: (398, 417)
Entity: Belgium | Type: nation | Span: (614, 621)
Entity: General Weygand | Type: person | Span: (736, 751)
Entity: General Gamelin | Type: person | Span: (781, 796)
Entity: French and British Armies | Type: military | Span: (824, 849)
Entity: Belgium | Type: nation | Span: (853, 860)
Entity: Belgians | Type: nation | Span: (902, 910)
Entity: French Army | Type: military | Span: (963, 974)
Entity: Somme | Type: location | Span: (1013, 1018)


/home/donbr/kgmgmt/kg_rememberall/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/donbr/kgmgmt/kg_rememberall/.venv/lib/python3.11/site-packages/glirel/model.py:704: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default valu


Extracted Relations:
Relation: ['General', 'Weygand', ','] -[commands]-> ['French', 'Army', 'which'] | Score: 0.659993588924408
Relation: ['General', 'Weygand', ','] -[commands]-> ['French', 'and', 'British', 'Armies', 'in'] | Score: 0.6598259806632996
Relation: ['General', 'Weygand', ','] -[commands]-> ['Armies', 'of', 'the', 'north', 'were'] | Score: 0.6482151746749878
Relation: ['French', 'and', 'British', 'Armies', 'in'] -[commands]-> ['General', 'Weygand', ','] | Score: 0.6403449177742004
Relation: ['Belgium', 'at'] -[commands]-> ['General', 'Weygand', ','] | Score: 0.6274427771568298
Relation: ['Belgium', '.'] -[commands]-> ['General', 'Weygand', ','] | Score: 0.626865804195404
Relation: ['Belgian', 'King', ';'] -[commands]-> ['General', 'Weygand', ','] | Score: 0.6187615394592285
Relation: ['French', 'Army', 'which'] -[commands]-> ['General', 'Weygand', ','] | Score: 0.6144334673881531
Relation: ['Armies', 'of', 'the', 'north', 'were'] -[commands]-> ['General', 'Weygand', ','] 

The upper case and abbreviated standard Named Entity Recognition (NER) labels, don't appear to work well as is.
- **"PERSON"** – People, including fictional characters.
- **"NORP"** – Nationalities, religious, or political groups.
- **"FAC"** – Buildings, airports, highways, bridges, etc.
- **"ORG"** – Organizations, such as companies, institutions, etc.
- **"GPE"** – Geopolitical entities, such as countries, cities, states.
- **"LOC"** – Non-GPE locations, such as mountains, bodies of water, etc.
- **"PRODUCT"** – Objects, vehicles, foods, etc. (not services).
- **"EVENT"** – Named hurricanes, battles, wars, sports events, etc.
- **"WORK_OF_ART"** – Titles of books, songs, movies, etc.
- **"LAW"** – Named documents made into laws.
- **"LANGUAGE"** – Any named language.
- **"DATE"** – Absolute or relative dates or periods.
- **"TIME"** – Times smaller than a day.
- **"PERCENT"** – Percentages, including "%".
- **"MONEY"** – Monetary values, including unit.
- **"QUANTITY"** – Measurements, such as weight or distance.
- **"ORDINAL"** – "first", "second", etc.
- **"CARDINAL"** – Numerals that do not fall under another type.

---

These labels are commonly used in **Named Entity Recognition (NER)** tasks to categorize entities in text. They are part of the standard NER framework in tools like **spaCy** and other NLP libraries.